<a href="https://colab.research.google.com/github/abdelilah-abdel/Deeplearning_projects/blob/main/Intelligent_Data_Analysis_Project_EuroSat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import pandas as pd
import cv2
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.utils import *

import tensorflow as tf
import tensorflow_datasets as tfds

%matplotlib inline

# Hide Warnings
# pd.options.mode.chained_assignment = None

# Set Seeds
tf.random.set_seed(42)
seed = 42
np.random.seed(42) # numpy seed (also used by keras)
random.seed(42)


# Intelligent Data Analysis 2 Project: EuroSAT

## Problem Setting

### Dataset

EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification (Vol. 12, Number 7, pp. 2217–2226). Zenodo. https://doi.org/10.5281/zenodo.7711810

EuroSAT is a land use and land cover classification dataset.

The dataset is based on Sentinel-2 satellite imagery covering 13 spectral bands and consists of 10 LULC classes with a total of 27,000 labeled and geo-referenced images. The RGB version of the dataset includes the optical R, G and B frequency bands encoded as JPEG images. The multi-spectral version of the EuroSAT dataset includes all 13 Sentinel-2 bands in the original value range. This project uses the RGB version.

### Task

Train a neural network that predicts the right class label (y) of 10 available labels for a given input image of size 64 x 64 px with the Channels R, G and B (X).



### Learning Problem

Multiclass image classification (supervised)

## Explorative Data Analysis

In [ ]:
ds, info = tfds.load('eurosat', split='all', data_dir='./data/eurosat', with_info=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

ds = tfds.as_numpy(ds)

images = []
labels = []

for image in ds:
    images.append(image['image'])
    labels.append(image['label'])

print(len(images))
print(len(labels))

images = np.array(images)
labels = np.array(labels)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
info.features

In [ ]:
# Labels
print("Number of Labels:", info.features["label"].num_classes)
print("Label Names:", info.features["label"].names)
# print(info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
# print(info.features["label"].str2int('AnnualCrop'))

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(14, 6))

label_id = 0
for x in range(0, 2):
        for y in range(0, 5):
            for image, label in zip(images, labels):
                if (label == label_id):
                        axs[x, y].imshow(image)
                        axs[x, y].set_title(str(label) + " " + info.features["label"].int2str(label))
                        axs[x, y].axis('off')
                        label_id += 1
                        break

## Preprocessing

In [ ]:
# train_test_split shuffles the data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=seed)
# X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.50, random_state=seed)

print(len(X_train))
print(len(X_test))
# print(len(X_validation))

In [ ]:
# Data Augmentation of the training data set
X_train_augmented = []
y_train_augmented = []

for image, label in zip(X_train, y_train):
    break
    # flip (cautious)
    # not implemented

    # rotate
    # img_90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    # img_180 = cv2.rotate(image, cv2.ROTATE_180)
    # img_270 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # X_train_augmented.append(img_90)
    # X_train_augmented.append(img_180)
    # X_train_augmented.append(img_270)

    # y_train_augmented.append(label)
    # y_train_augmented.append(label)
    # y_train_augmented.append(label)

X_train_complete = np.array(list(X_train) + X_train_augmented)
print(len(X_train_complete))

y_train_complete = np.array(list(y_train) + y_train_augmented)
print(len(y_train_complete))

In [ ]:
# Plot distribution of class labels in training and test set
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
sns.histplot(ax=axes[0], data=y_train_complete, bins=10)
axes[0].set_title("Class Distribution in y_training")
axes[0].set_xticks(ticks=range(0, 10), labels=info.features["label"].names, rotation=90)

sns.histplot(ax=axes[1], data=y_test, bins=10)
axes[1].set_title("Class Distribution in y_test")
axes[1].set_xticks(ticks=range(0, 10), labels=info.features["label"].names, rotation=90)

fig.tight_layout()

In [ ]:
# normalize images
min_val = np.amin(X_train_complete)
max_val = np.amax(X_train_complete)
X_train_rescaled = (X_train_complete.astype(float) - min_val) / (max_val - min_val)
print("(x_train) min_val:", min_val, "max_val:", max_val)

min_val = np.amin(X_test)
max_val = np.amax(X_test)
X_test_rescaled = (X_test.astype(float) - min_val) / (max_val - min_val)
print("(x_test) min_val:", min_val, "max_val:", max_val)

# Print comparison of original and rescaled image
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axes[0].imshow(X_train_complete[0])
axes[0].title.set_text('Original')
axes[0].axis('off')

axes[1].imshow(X_train_rescaled[0])
axes[1].title.set_text('Rescaled')
axes[1].axis('off')

fig.tight_layout()

In [ ]:
X_test_rescaled.shape

In [ ]:
from keras.utils import to_categorical

In [ ]:
nb_classes = 10
oneHotLabelTrain = to_categorical(y_train_complete, nb_classes)
oneHotLabelTest  = to_categorical(y_test,  nb_classes)
# oneHotLabelValidation  = np_utils.to_categorical(y_validation,  nb_classes)

## Method Application

### CNN based on exercise (baseline)

In [ ]:
# simple CNN
def getBaselineCNNModel(
    input_shape = (64, 64, 3),
    nb_filters_one = 32,
    nb_filters_two = 64,
    nb_conv = 3,
    nb_pool = 2,
    dense_size = 128
):

    cnnModel = Sequential()
    cnnModel.add(Conv2D(nb_filters_one, kernel_size=(nb_conv, nb_conv),
                     activation='relu',
                     input_shape=input_shape)) # convolutional layer
    cnnModel.add(MaxPooling2D(pool_size=(nb_pool, nb_pool))) # max pooling layer
    cnnModel.add(Dropout(0.25))
    cnnModel.add(Flatten())
    cnnModel.add(Dense(dense_size, activation='relu')) # dense layer
    cnnModel.add(Dropout(0.5))
    cnnModel.add(Dense(nb_classes, activation='softmax'))

    cnnModel.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return cnnModel

cnnModel = getBaselineCNNModel()
cnnModel.summary()

In [ ]:
batch_size = 64
numEpochs = 10

cnnModel = getBaselineCNNModel()
learnHistCNN = cnnModel.fit(X_train_rescaled,oneHotLabelTrain,
                               validation_data=(X_test_rescaled,oneHotLabelTest),
                               batch_size=batch_size,
                               epochs=numEpochs)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axes[0].plot(learnHistCNN.history['loss'], label="loss")
axes[0].plot(learnHistCNN.history['val_loss'], label="validation_loss")
axes[0].title.set_text('Loss')
axes[0].legend(loc="upper right")

axes[1].plot(learnHistCNN.history['accuracy'], label="accuracy")
axes[1].plot(learnHistCNN.history['val_accuracy'], label="val_accuracy")
axes[1].title.set_text('Accuracy')
axes[1].legend(loc="upper right")

fig.tight_layout()

In [ ]:
preds = cnnModel.predict(X_test_rescaled)
class_labels = np.argmax(preds, axis=1)

cnnModelCM = ConfusionMatrixDisplay.from_predictions(y_test,class_labels,display_labels=info.features["label"].names,xticks_rotation='vertical')

In [ ]:
import pickle

# Create and train model...

# Serialize entire model to .pkl file
with open('model1.pkl', 'wb') as f:
  pickle.dump(cnnModel, f)


### CNN (more advanced)

In [ ]:
# Advanced CNN
def getAdvancedCNNModel(
    input_shape = (64, 64, 3),
    nb_filters_one = 32,
    nb_filters_two = 64,
    nb_filters_three = 128,
    nb_conv = 3,
    nb_pool = 2,
    dense_size = 256
):

    cnnModel = Sequential()

    cnnModel.add(Conv2D(nb_filters_one, kernel_size=(nb_conv, nb_conv),
                     activation='relu',
                     input_shape=input_shape)) # convolutional layer
    cnnModel.add(MaxPooling2D(pool_size=(nb_pool, nb_pool))) # max pooling layer

    # make training faster and more stable through
    # normalization of the layers' inputs by re-centering and re-scaling
    # cnnModel.add(BatchNormalization())

    cnnModel.add(Conv2D(nb_filters_two, kernel_size=(nb_conv, nb_conv),
                     activation='relu',
                     input_shape=input_shape)) # convolutional layer
    cnnModel.add(MaxPooling2D(pool_size=(nb_pool, nb_pool))) # max pooling layer

    cnnModel.add(Conv2D(nb_filters_three, kernel_size=(nb_conv, nb_conv),
                     activation='relu',
                     input_shape=input_shape)) # convolutional layer
    cnnModel.add(MaxPooling2D(pool_size=(nb_pool, nb_pool))) # max pooling layer

    # cnnModel.add(BatchNormalization())
    cnnModel.add(Dropout(0.25))

    cnnModel.add(Flatten())
    cnnModel.add(Dense(dense_size, activation='relu')) # dense layer
    # cnnModel.add(BatchNormalization())
    cnnModel.add(Dropout(0.5))

    cnnModel.add(Dense(nb_classes, activation='softmax'))

    cnnModel.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return cnnModel

cnnModelAdv = getAdvancedCNNModel()
cnnModelAdv.summary()

In [ ]:
batch_size = 64
numEpochs = 10

cnnModelAdv = getAdvancedCNNModel()
learnHistCNNAdv = cnnModelAdv.fit(X_train_rescaled,oneHotLabelTrain,
                               validation_data=(X_test_rescaled,oneHotLabelTest),
                               batch_size=batch_size,
                               epochs=numEpochs)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axes[0].plot(learnHistCNNAdv.history['loss'], label="loss")
axes[0].plot(learnHistCNNAdv.history['val_loss'], label="validation_loss")
axes[0].title.set_text('Loss')
axes[0].legend(loc="upper right")

axes[1].plot(learnHistCNNAdv.history['accuracy'], label="accuracy")
axes[1].plot(learnHistCNNAdv.history['val_accuracy'], label="val_accuracy")
axes[1].title.set_text('Accuracy')
axes[1].legend(loc="upper right")

fig.tight_layout()

In [ ]:
with open('model2.pkl', 'wb') as f:
  pickle.dump(cnnModelAdv, f)

In [ ]:
preds = cnnModelAdv.predict(X_test_rescaled)
class_labels = np.argmax(preds, axis=1)

cnnModelAdvCM = ConfusionMatrixDisplay.from_predictions(y_test,
                                            class_labels,
                                            display_labels=info.features["label"].names,
                                            xticks_rotation='vertical')

### ResNet (transfer learning)

In [ ]:
def getResNet50():
    nb_classes = 10
    dense_size = 128

    base_model = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (64,64,3), pooling='avg')

    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(dense_size, activation='relu')(x)
    x = (Dropout(0.5))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)

    head_model = Model(inputs = base_model.input, outputs = predictions)

    head_model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

    return head_model

In [ ]:
resnetmodel = getResNet50()
resnetmodel.summary()

In [ ]:
batch_size = 32
numEpochs = 10

historyRes = resnetmodel.fit(X_train_complete, oneHotLabelTrain, batch_size=batch_size, epochs=numEpochs,
                         validation_data=(X_test, oneHotLabelTest))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axes[0].plot(historyRes.history['loss'], label="loss")
axes[0].plot(historyRes.history['val_loss'], label="validation_loss")
axes[0].title.set_text('Loss')
axes[0].legend(loc="upper right")

axes[1].plot(historyRes.history['accuracy'], label="accuracy")
axes[1].plot(historyRes.history['val_accuracy'], label="val_accuracy")
axes[1].title.set_text('Accuracy')
axes[1].legend(loc="upper right")

fig.tight_layout()

In [ ]:
with open('model3.pkl', 'wb') as f:
  pickle.dump(resnetmodel, f)

In [ ]:
preds = resnetmodel.predict(X_test)
class_labels = np.argmax(preds, axis=1)

resnetmodelCM = ConfusionMatrixDisplay.from_predictions(y_test,
                                            class_labels,
                                            display_labels=info.features["label"].names,
                                            xticks_rotation='vertical')


## Experiments

In [ ]:
!pip install keras-tuner --upgrade

from keras_tuner import RandomSearch

def getHyperparameterBaselineCNNModel(hp):
    nb_filters_one = 32,
    nb_filters_two = 64,

    # hyperparameters
    nb_conv = hp.Int('nb_conv', min_value=3, max_value=7, step=2)
    dense_size = hp.Int('dense_size', min_value=32, max_value=128, step=16)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    cnnModel = Sequential()
    cnnModel.add(Conv2D(32, kernel_size=(nb_conv, nb_conv), activation='relu', input_shape=(64, 64, 3))) # convolutional layer
    cnnModel.add(MaxPooling2D(pool_size=(2, 2))) # max pooling layer
    cnnModel.add(Dropout(0.25))
    cnnModel.add(Flatten())
    cnnModel.add(Dense(dense_size, activation='relu')) # dense layer
    cnnModel.add(Dropout(0.5))
    cnnModel.add(Dense(nb_classes, activation='softmax'))

    cnnModel.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=hp_learning_rate), metrics=['accuracy'])

    return cnnModel

# create randomsearch object
tuner = RandomSearch(hypermodel=getHyperparameterBaselineCNNModel,
                    objective = 'val_accuracy',
                    max_trials = 10)

# search for best parameter
tuner.search(X_train_rescaled,
             oneHotLabelTrain,
             epochs=4,
             validation_data=(X_test_rescaled,oneHotLabelTest),)

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("dense_size", best_hps.get('dense_size'))
print("nb_conv", best_hps.get('nb_conv'))
print("learning_rate", best_hps.get('learning_rate'))

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
modelHyper = tuner.hypermodel.build(best_hps)
historyHP = modelHyper.fit(X_train_rescaled, oneHotLabelTrain, epochs=10, validation_split=0.2)

In [ ]:
modelHyper.summary()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
axes[0].plot(historyHP.history['loss'], label="loss")
axes[0].plot(historyHP.history['val_loss'], label="validation_loss")
axes[0].title.set_text('Loss')
axes[0].legend(loc="upper right")

axes[1].plot(historyHP.history['accuracy'], label="accuracy")
axes[1].plot(historyHP.history['val_accuracy'], label="val_accuracy")
axes[1].title.set_text('Accuracy')
axes[1].legend(loc="upper right")

fig.tight_layout()

In [ ]:
preds = modelHyper.predict(X_test_rescaled)
class_labels = np.argmax(preds, axis=1)

modelHyperCM = ConfusionMatrixDisplay.from_predictions(y_test,
                                            class_labels,
                                            display_labels=info.features["label"].names,
                                            xticks_rotation='vertical')

## Conclusions


In [ ]:
# Comparison of models
# df = pd.DataFrame(columns=['Total Params', 'Trainable Params', 'Non-Trainable Params', 'Loss', 'Val_Loss', 'Accuracy', 'Val_Accuracy',])

# baseline - learnHistCNN - cnnModel
cnnModelTrainable = np.sum([K.count_params(w) for w in cnnModel.trainable_weights])
cnnModelNonTrainable = np.sum([K.count_params(w) for w in cnnModel.non_trainable_weights])
cnnModelTotal = cnnModelTrainable + cnnModelNonTrainable
cnnModelLoss = learnHistCNN.history['loss'][-1]
cnnModelValLoss = learnHistCNN.history['val_loss'][-1]
cnnModelAccuracy = learnHistCNN.history['accuracy'][-1]
cnnModelValAccuracy = learnHistCNN.history['val_accuracy'][-1]

# advanced - learnHistCNNAdv - cnnModelAdv
cnnModelAdvTrainable = np.sum([K.count_params(w) for w in cnnModelAdv.trainable_weights])
cnnModelAdvNonTrainable = np.sum([K.count_params(w) for w in cnnModelAdv.non_trainable_weights])
cnnModelAdvTotal = cnnModelAdvTrainable + cnnModelAdvNonTrainable
cnnModelAdvLoss = learnHistCNNAdv.history['loss'][-1]
cnnModelAdvValLoss = learnHistCNNAdv.history['val_loss'][-1]
cnnModelAdvAccuracy = learnHistCNNAdv.history['accuracy'][-1]
cnnModelAdvValAccuracy = learnHistCNNAdv.history['val_accuracy'][-1]

# resnet - historyRes - resnetmodel
resnetmodelTrainable = np.sum([K.count_params(w) for w in resnetmodel.trainable_weights])
resnetmodelNonTrainable = np.sum([K.count_params(w) for w in resnetmodel.non_trainable_weights])
resnetmodelTotal = resnetmodelTrainable + resnetmodelNonTrainable
resnetmodelLoss = historyRes.history['loss'][-1]
resnetmodelValLoss = historyRes.history['val_loss'][-1]
resnetmodelAccuracy = historyRes.history['accuracy'][-1]
resnetmodelValAccuracy = historyRes.history['val_accuracy'][-1]

# hyperparameter baseline - historyHP - modelHyper
modelHyperTrainable = np.sum([K.count_params(w) for w in modelHyper.trainable_weights])
modelHyperNonTrainable = np.sum([K.count_params(w) for w in modelHyper.non_trainable_weights])
modelHyperTotal = modelHyperTrainable + modelHyperNonTrainable
modelHyperLoss = historyHP.history['loss'][-1]
modelHyperValLoss = historyHP.history['val_loss'][-1]
modelHyperAccuracy = historyHP.history['accuracy'][-1]
modelHyperValAccuracy = historyHP.history['val_accuracy'][-1]

df = pd.DataFrame({'Name': ["Baseline CNN","Advanced CNN","ResNet","Baseline CNN (HP)"],
                   'Total Params' : [cnnModelTotal,cnnModelAdvTotal,resnetmodelTotal,modelHyperTotal],
                   'Trainable Params' : [cnnModelTrainable,cnnModelAdvTrainable,resnetmodelTrainable,modelHyperTrainable],
                   'Non-Trainable Params' : [cnnModelNonTrainable,cnnModelAdvNonTrainable,resnetmodelNonTrainable,modelHyperNonTrainable],
                   'Loss' : [cnnModelLoss,cnnModelAdvLoss,resnetmodelLoss,modelHyperLoss],
                   'Val_Loss' : [cnnModelValLoss,cnnModelAdvValLoss,resnetmodelValLoss,modelHyperValLoss],
                   'Accuracy' : [cnnModelAccuracy,cnnModelAdvAccuracy,resnetmodelAccuracy,modelHyperAccuracy],
                   'Val_Accuracy' : [cnnModelValAccuracy,cnnModelAdvValAccuracy,resnetmodelValAccuracy,modelHyperValAccuracy]})

# format
df['Total Params'] = df['Total Params'].astype('int64')
df['Trainable Params'] = df['Trainable Params'].astype('int64')
df['Non-Trainable Params'] = df['Non-Trainable Params'].astype('int64')
pd.options.display.float_format = '{:.4f}'.format
display(df)

In [ ]:
# Comparison of models
# baseline - learnHistCNN - cnnModel
# advanced - learnHistCNNAdv - cnnModelAdv
# resnet - historyRes - resnetmodel
# hyperparameter baseline - historyHP - modelHyper

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
axes[0].plot(learnHistCNN.history['loss'], label="(baseline) loss")
axes[0].plot(learnHistCNN.history['val_loss'], label="(baseline) val_loss")
axes[0].plot(learnHistCNNAdv.history['loss'], label="(advanced) loss")
axes[0].plot(learnHistCNNAdv.history['val_loss'], label="(advanced) val_loss")
axes[0].plot(historyRes.history['loss'], label="(resnet) loss")
axes[0].plot(historyRes.history['val_loss'], label="(resnet) val_loss")
axes[0].plot(historyHP.history['loss'], label="(hp) loss")
axes[0].plot(historyHP.history['val_loss'], label="(hp) val_loss")
axes[0].title.set_text('Loss')
axes[0].legend(loc="lower right")

axes[1].plot(learnHistCNN.history['accuracy'], label="(baseline) accuracy")
axes[1].plot(learnHistCNN.history['val_accuracy'], label="(baseline) val_accuracy")
axes[1].plot(learnHistCNNAdv.history['accuracy'], label="(advanced) accuracy")
axes[1].plot(learnHistCNNAdv.history['val_accuracy'], label="(advanced) val_accuracy")
axes[1].plot(historyRes.history['accuracy'], label="(resnet) accuracy")
axes[1].plot(historyRes.history['val_accuracy'], label="(resnet) val_accuracy")
axes[1].plot(historyHP.history['accuracy'], label="(hp) accuracy")
axes[1].plot(historyHP.history['val_accuracy'], label="(hp) val_accuracy")
axes[1].title.set_text('Accuracy')
axes[1].legend(loc="lower right")

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(14, 6))
cnnModelCM.plot(ax=axes[0])

axes[0].title.set_text('Baseline CNN')
axes[0].axis('off')

cnnModelAdvCM.plot(ax=axes[1])
axes[1].title.set_text('Advanced CNN')
axes[1].axis('off')

resnetmodelCM.plot(ax=axes[2])
axes[2].title.set_text('ResNet')
axes[2].axis('off')

modelHyperCM.plot(ax=axes[3])
axes[3].title.set_text('Baseline CNN (HP)')
axes[3].axis('off')

fig.tight_layout()

In [ ]:
# ResNet outperforms all other models by far
# Differences between baseline, advanced and hyper-parameter tuned CNN are not noticable

In [ ]:
# Future Tests: Vision Transformers (see Dosovitskiy, A., Beyer, L., Kolesnikov, A., Weissenborn, D., Zhai, X., Unterthiner, T., ... & Houlsby, N. (2020). An image is worth 16x16 words: Transformers for image recognition at scale. arXiv preprint arXiv:2010.11929)
# Autoencoder also possible but not the best option
# - Vision Transformers https://keras.io/examples/vision/image_classification_with_vision_transformer/